In [1]:
import sys
sys.path.append(r"C:\Users\shrey")

In [31]:
import numpy as np
import matplotlib.pyplot as plt
import json
import pandas as pd

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.converters import circuit_to_dag
from qiskit.transpiler import Target
from qiskit.circuit.library import QFT
from qiskit.circuit import Delay
from qiskit.circuit.library import XGate, YGate, ZGate
from qiskit.circuit import Gate
from qiskit.circuit.random import random_circuit
from qiskit.quantum_info import hellinger_fidelity

from analytics.qubit_idling import analyze_qubit_idling, analyze_qubit_activity
from analytics.backend_characterization import extract_backend_metrics

In [32]:
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService(name="Reserach_IITJ")
backend = service.backend("ibm_torino")

management.get:WARNING:2025-12-07 12:52:33,923: Loading saved account: Reserach_IITJ


In [33]:
from qiskit import QuantumCircuit
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import DynamicalDecoupling
from qiskit.circuit.library import XGate
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

# --- 1️⃣ Normal preset transpiler ---
pm = generate_preset_pass_manager(
    optimization_level=3,
    seed_transpiler=42,
    backend=backend,
    scheduling_method='asap'
)

In [34]:
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes.scheduling import ASAPScheduleAnalysis
from qiskit.transpiler.passes import PadDynamicalDecoupling
from qiskit.transpiler import InstructionDurations
from qiskit.circuit.library import XGate

DD_SEQUENCE = [XGate(), XGate().inverse()]

DD_PM = PassManager(
    [
        ASAPScheduleAnalysis(backend.instruction_durations),
        PadDynamicalDecoupling(
            durations=backend.instruction_durations, dd_sequence=DD_SEQUENCE
        ),
    ]
)


def dynamical_decoupling_preprocess(
    input_circuit: QuantumCircuit, dd_pass_manager=DD_PM
) -> QuantumCircuit:
    """Apply dynamical decoupling to the input circuit.

    Args:
        input_circuit (QuantumCircuit): input circuit to run error mitigation on.
    """
    return dd_pass_manager.run(input_circuit)

In [35]:
from qiskit_aer import AerSimulator
simulator = AerSimulator()
backend_sim = AerSimulator.from_backend(backend)

In [36]:
from qiskit_ibm_runtime import SamplerV2, SamplerOptions

sampler_options = SamplerOptions()
sampler_options.dynamical_decoupling.enable = False
sampler_options.twirling.enable_gates = False
sampler_options.twirling.enable_measure = False

sampler_no_dd = SamplerV2(mode = backend, options = sampler_options)


In [37]:
sampler_options = SamplerOptions()
sampler_options.dynamical_decoupling.enable = True
sampler_options.twirling.enable_gates = False
sampler_options.twirling.enable_measure = False

sampler_dd = SamplerV2(mode = backend, options = sampler_options)

In [38]:
all_circuits_df = pd.DataFrame(columns=[
    "circuit_id", "qubit",
    "sparsity_by_layer", "sparsity_by_time",
    "mean_dt", "sd_dt",
    "max_delay_dt", "decoher_err_prob",
    "marginal_fid_no_dd", "marginal_fid_dd", "marginal_improvment",
    "hellinger_fidelity_no_dd", "hellinger_fidelity_dd", "overall_improvment"
])

# --- Function to store metrics for one circuit ---
def store_circuit_results(qubit_act, qubit_idle, marginal_fid_no_dd, marginal_fid_dd, hellinger_fid_no_dd, hellinger_fid_dd, circuit_id):
    rows = []
    for qubit in qubit_act.get("sparsity_by_layer", {}).keys():
        row = {
            "circuit_id": circuit_id,
            "qubit": qubit,
            # From qubit activity
            "sparsity_by_layer": qubit_act["sparsity_by_layer"].get(qubit, None),
            "sparsity_by_time": qubit_act.get("sparsity_by_time", {}).get(qubit, None),
            "mean_dt": qubit_act.get("distribution_mean_dt", {}).get(qubit, None),
            "sd_dt": qubit_act.get("distribution_sd_dt", {}).get(qubit, None),
            # From qubit idling (decoherence)
            "max_delay_dt": qubit_idle["overall_average"].get("max_delay_dt", None),
            "decoher_err_prob": qubit_idle["overall_average"].get("decoher_err_prob", None),
            "marginal_fid_no_dd": marginal_fid_no_dd.get(qubit, None),
            "marginal_fid_dd": marginal_fid_dd.get(qubit, None),
            "marginal_improvment": marginal_fid_dd.get(qubit, 0) - marginal_fid_no_dd.get(qubit, 0),
            # Hellinger fidelity
            "hellinger_fidelity_no_dd": hellinger_fid_no_dd,
            "hellinger_fidelity_dd": hellinger_fid_dd,
            "overall_improvment": hellinger_fid_dd - hellinger_fid_no_dd
        }
        rows.append(row)
    return pd.DataFrame(rows)

In [39]:
from qiskit.result import marginal_distribution

def get_marginal_distributions(prob_dist, n):
    marginals = {}
    for qubit in range(n):
        bit_index = n - 1 - qubit
        marginals[qubit] = marginal_distribution(prob_dist, [bit_index])

    return marginals

In [40]:
def ghz_circuit(num_qubits, measure=True):
    """
    Generate a GHZ state circuit with num_qubits qubits.
    If measure=True, adds measurements to all qubits.
    """
    qc = QuantumCircuit(num_qubits, num_qubits if measure else 0)
    
    # Step 1: Apply H on the first qubit
    qc.h(0)
    
    # Step 2: Apply a chain of CX gates
    for i in range(num_qubits - 1):
        qc.cx(i, i + 1)
    
    qc.barrier()

    # Step 3: Optional measurement
    if measure:
        qc.measure(range(num_qubits), range(num_qubits))
    
    return qc

def echo_circuit(n):
    # Create GHZ state
    ghz = QuantumCircuit(n, n)
    ghz.h(0)
    for i in range(n - 1):
        ghz.cx(i, i + 1)

    # Loschmidt echo circuit: GHZ → barrier → GHZ† → measurement
    echo = QuantumCircuit(n)
    echo.compose(ghz, inplace=True)
    echo.barrier()
    echo.compose(ghz.inverse(), inplace=True)
    echo.x(range(n)) 
    echo.measure(range(n), range(n))

    return echo

In [41]:
def compute_fidelity(num_qubits, quantum_circuit, name):
    transpiled_circ = pm.run(quantum_circuit)
    transpiled_circ_dd = dynamical_decoupling_preprocess(transpiled_circ)

    backend_prop = extract_backend_metrics(transpiled_circ, backend)
    qubit_act = analyze_qubit_activity(transpiled_circ, backend.target)
    qubit_idle = analyze_qubit_idling(transpiled_circ, backend)

    # --- Run Sampler and Estimator (example) ---
    idle_counts = simulator.run(transpiled_circ).result().get_counts()
    result_sampler_no_dd = sampler_no_dd.run([transpiled_circ]).result()[0].data.c.get_counts()
    result_sampler_dd = sampler_dd.run([transpiled_circ_dd]).result()[0].data.c.get_counts()

    # --- Compare Sampler results using Hellinger fidelity ---
    hellinger_fid_no_dd = hellinger_fidelity(idle_counts, result_sampler_no_dd)
    hellinger_fid_dd = hellinger_fidelity(idle_counts, result_sampler_dd)

    # Calculate indiviual qubit marginals
    marginal_dist_idle = get_marginal_distributions(idle_counts, num_qubits)
    marginal_dist_no_dd = get_marginal_distributions(result_sampler_no_dd, num_qubits)
    marginal_dist_dd = get_marginal_distributions(result_sampler_dd, num_qubits)
        
    layout = transpiled_circ.layout.final_index_layout()
    marginal_fid_no_dd = {}
    marginal_fid_dd = {}

    for i in range(num_qubits):
        marginal_fid_no_dd[layout[i]] = hellinger_fidelity(
            marginal_dist_idle[i],
            marginal_dist_no_dd[i]
        )
        marginal_fid_dd[layout[i]] = hellinger_fidelity(
            marginal_dist_idle[i],
            marginal_dist_dd[i]
        )

    df_first_circuit = store_circuit_results(qubit_act, qubit_idle, marginal_fid_no_dd, marginal_fid_dd, hellinger_fid_no_dd, hellinger_fid_dd, name)
    return df_first_circuit

In [42]:
num_qubits_list = [2,7, 12, 15, 30]
for num_qubits in num_qubits_list:
    rand_circ = ghz_circuit(num_qubits)
    df_first_circuit = compute_fidelity(num_qubits, rand_circ, f"GHZ_{num_qubits}")
    all_circuits_df = pd.concat([all_circuits_df, df_first_circuit], ignore_index=True)

KeyboardInterrupt: 

In [ ]:
num_qubits_list = [2,7, 12, 15,30]
for num_qubits in num_qubits_list:
    rand_circ = echo_circuit(num_qubits)
    df_first_circuit = compute_fidelity(num_qubits, rand_circ, f"Echo_{num_qubits}")
    all_circuits_df = pd.concat([all_circuits_df, df_first_circuit], ignore_index=True)

KeyboardInterrupt: 

In [ ]:
all_circuits_df.to_csv("E:\\Desktop\\QAMP results\\Torino_with_sampler_071225_asap.csv")